In [1]:
!pip install evaluate accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 84.1/84.1 kB 2.0 MB/s eta 0:00:00


In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [3]:
from transformers import AutoModelForSequenceClassification, AutoTokenizer, TrainingArguments, Trainer
import torch
from sklearn.utils.class_weight import compute_class_weight
import numpy as np
from datasets import load_dataset

dataset = load_dataset("banking77")

/usr/local/lib/python3.12/dist-packages/huggingface_hub/utils/_auth.py:94: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


README.md: 0.00B [00:00, ?B/s]

data/train-00000-of-00001.parquet:   0%|          | 0.00/298k [00:00<?, ?B/s]

data/test-00000-of-00001.parquet:   0%|          | 0.00/93.9k [00:00<?, ?B/s]

Generating train split:   0%|          | 0/10003 [00:00<?, ? examples/s]

Generating test split:   0%|          | 0/3080 [00:00<?, ? examples/s]

In [4]:
from datasets import load_from_disk
import json

# Load tokenized datasets
model_path = "/content/drive/My Drive/Colab Notebooks/Week 8 banking77/models/banking77-classifier"
dataset_save_path ="/content/drive/My Drive/Colab Notebooks/Week 8 banking77/models/banking77-classifier/tokenized_datasets"
tokenized_datasets = load_from_disk(dataset_save_path)
tokenizer = AutoTokenizer.from_pretrained(model_path)

# Load label mappings
with open(f"{dataset_save_path}/label_mappings.json", 'r') as f:
    label_mappings = json.load(f)

id2label = label_mappings['id2label']
label2id = label_mappings['label2id']

print(f"✅ Loaded tokenized dataset: {len(tokenized_datasets['train'])} training examples")
print(f"✅ Label mappings loaded: {len(id2label)} classes")

✅ Loaded tokenized dataset: 10003 training examples
✅ Label mappings loaded: 77 classes


In [5]:
print("=== Model Initialization with Class Weights ===")

# Create label mappings
id2label = {i: label for i, label in enumerate(dataset['train'].features['label'].names)}
label2id = {label: i for i, label in enumerate(dataset['train'].features['label'].names)}

print(f"Sample label mapping: {list(id2label.items())[:5]}")

# 🆕 CALCULATE CLASS WEIGHTS
print("\n=== Calculating Class Weights ===")
train_labels = dataset['train']['label']
class_weights = compute_class_weight(
    'balanced',
    classes=np.unique(train_labels),
    y=train_labels
)

class_weights = torch.tensor(class_weights, dtype=torch.float32)
print(f"Class weights calculated: {len(class_weights)} classes")
print(f"Weight range: {class_weights.min():.3f} to {class_weights.max():.3f}")

# Load the model
model = AutoModelForSequenceClassification.from_pretrained(
    "distilbert-base-uncased",
    num_labels=77,
    id2label=id2label,
    label2id=label2id
)

print(f"\n=== Model Details ===")
print(f"Model type: {type(model)}")
print(f"Number of parameters: {model.num_parameters():,}")
print(f"Model device: {next(model.parameters()).device}")

# Test forward pass
print("\n=== Testing Forward Pass ===")
test_input = tokenized_datasets['train'][:2]  # First 2 examples
test_input = {k: torch.tensor(v) for k, v in test_input.items()}

with torch.no_grad():
    outputs = model(**test_input)

print(f"Output logits shape: {outputs.logits.shape}")
print(f"Loss: {outputs.loss if outputs.loss else 'N/A'}")

=== Model Initialization with Class Weights ===
Sample label mapping: [(0, 'activate_my_card'), (1, 'age_limit'), (2, 'apple_pay_or_google_pay'), (3, 'atm_support'), (4, 'automatic_top_up')]

=== Calculating Class Weights ===
Class weights calculated: 77 classes
Weight range: 0.695 to 3.712


config.json:   0%|          | 0.00/483 [00:00<?, ?B/s]

model.safetensors:   0%|          | 0.00/268M [00:00<?, ?B/s]

Some weights of DistilBertForSequenceClassification were not initialized from the model checkpoint at distilbert-base-uncased and are newly initialized: ['classifier.bias', 'classifier.weight', 'pre_classifier.bias', 'pre_classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.



=== Model Details ===
Model type: <class 'transformers.models.distilbert.modeling_distilbert.DistilBertForSequenceClassification'>
Number of parameters: 67,012,685
Model device: cpu

=== Testing Forward Pass ===
Output logits shape: torch.Size([2, 77])
Loss: 4.238710880279541


In [6]:
# 🆕 CUSTOM TRAINER WITH CLASS WEIGHTS
class WeightedTrainer(Trainer):
    def compute_loss(self, model, inputs, return_outputs=False, num_items_in_batch=None):
        labels = inputs.get("labels")
        outputs = model(**inputs)
        logits = outputs.get("logits")

        # Move class weights to the same device as logits
        device = logits.device
        weights = class_weights.to(device)

        # Use class weights in loss calculation
        loss_fct = torch.nn.CrossEntropyLoss(weight=weights)
        loss = loss_fct(logits.view(-1, self.model.config.num_labels), labels.view(-1))

        return (loss, outputs) if return_outputs else loss

print("✓ WeightedTrainer class defined")

✓ WeightedTrainer class defined


In [7]:
import evaluate

# Load evaluation metrics
accuracy_metric = evaluate.load("accuracy")
f1_metric = evaluate.load("f1")

def compute_metrics(eval_pred):
    """Compute accuracy and F1 score for evaluation"""
    predictions, labels = eval_pred
    predictions = np.argmax(predictions, axis=1)

    accuracy = accuracy_metric.compute(predictions=predictions, references=labels)
    f1 = f1_metric.compute(predictions=predictions, references=labels, average="weighted")

    return {"accuracy": accuracy["accuracy"], "f1": f1["f1"]}

# Configure training
training_args = TrainingArguments(
    output_dir="./banking77-classifier-weighted",
    # Training parameters
    learning_rate=2e-5,
    per_device_train_batch_size=16,
    per_device_eval_batch_size=16,
    num_train_epochs=3,
    weight_decay=0.01,

    # Evaluation & saving
    eval_strategy="epoch",
    save_strategy="epoch",
    load_best_model_at_end=True,
    metric_for_best_model="f1",

    # Logging
    logging_dir="./logs",
    logging_steps=50,
    report_to=None,

    # Optimization
    warmup_steps=100,
    save_total_limit=2,
)

print("=== Training Configuration ===")
print(f"Total training steps: {len(tokenized_datasets['train']) // 16 * 3}")
print(f"Batch size: {training_args.per_device_train_batch_size}")
print(f"Learning rate: {training_args.learning_rate}")
print(f"Using class weights: ✅ YES")

# 🆕 USE WEIGHTED TRAINER
trainer = WeightedTrainer(
    model=model,
    args=training_args,
    train_dataset=tokenized_datasets["train"],
    eval_dataset=tokenized_datasets["test"],
    tokenizer=tokenizer,
    compute_metrics=compute_metrics,
)

print("\n=== Trainer Ready with Class Weights ===")
print(f"Training dataset size: {len(tokenized_datasets['train'])}")
print(f"Evaluation dataset size: {len(tokenized_datasets['test'])}")

=== Training Configuration ===
Total training steps: 1875
Batch size: 16
Learning rate: 2e-05
Using class weights: ✅ YES


/tmp/ipython-input-1231132921.py:50: FutureWarning: `tokenizer` is deprecated and will be removed in version 5.0.0 for `WeightedTrainer.__init__`. Use `processing_class` instead.
  trainer = WeightedTrainer(



=== Trainer Ready with Class Weights ===
Training dataset size: 10003
Evaluation dataset size: 3080


In [8]:
print("=== Starting Training with Class Weights ===")

# Train the model
train_result = trainer.train()

# Save training metrics
trainer.save_model("./banking77-classifier-weighted-final")
trainer.log_metrics("train", train_result.metrics)
trainer.save_metrics("train", train_result.metrics)
trainer.save_state()

print("✓ Training completed with class weights!")
print(f"Final training loss: {train_result.metrics.get('train_loss', 'N/A')}")

=== Starting Training with Class Weights ===


/usr/local/lib/python3.12/dist-packages/notebook/notebookapp.py:191: SyntaxWarning: invalid escape sequence '\/'
  | |_| | '_ \/ _` / _` |  _/ -_)


<IPython.core.display.Javascript object>

wandb: Logging into wandb.ai. (Learn how to deploy a W&B server locally: https://wandb.me/wandb-server)
wandb: You can find your API key in your browser here: https://wandb.ai/authorize?ref=models
wandb: Paste an API key from your profile and hit enter:

 ··········


wandb: WARNING If you're specifying your api key in code, ensure this code is not shared publicly.
wandb: WARNING Consider setting the WANDB_API_KEY environment variable, or running `wandb login` from the command line.
wandb: No netrc file found, creating one.
wandb: Appending key for api.wandb.ai to your netrc file: /root/.netrc
wandb: Currently logged in as: zhanghanxue15 (zhanghanxue15-personal) to https://api.wandb.ai. Use `wandb login --relogin` to force relogin


Epoch,Training Loss,Validation Loss,Accuracy,F1
1,2.328400,2.016145,0.714286,0.691511
2,1.175400,1.028501,0.842208,0.839332
3,0.911700,0.816547,0.867208,0.866431


***** train metrics *****
  epoch                    =        3.0
  total_flos               =   701266GF
  train_loss               =     1.9384
  train_runtime            = 0:07:44.84
  train_samples_per_second =     64.556
  train_steps_per_second   =       4.04
✓ Training completed with class weights!
Final training loss: 1.938359806966731


In [9]:
print("=== Comprehensive Evaluation ===")

# Evaluate on test set
eval_results = trainer.evaluate()
print("\n📊 Evaluation Results:")
for key, value in eval_results.items():
    print(f"  {key}: {value:.4f}")

# 🆕 PER-CLASS PERFORMANCE ANALYSIS
from sklearn.metrics import classification_report, confusion_matrix

# Get predictions for detailed analysis
predictions = trainer.predict(tokenized_datasets["test"])
predicted_labels = np.argmax(predictions.predictions, axis=1)
true_labels = predictions.label_ids

print("\n=== Per-Class Performance ===")
class_report = classification_report(
    true_labels,
    predicted_labels,
    target_names=[id2label[i] for i in range(len(id2label))],  # Provide all class names
    output_dict=False
)
print(class_report)

# 📝 TEST PREDICTIONS WITH CONFIDENCE
print("\n=== Sample Predictions ===")
test_samples = [
    "I need to check my account balance",
    "My card was stolen what should I do",
    "I want to apply for a loan",
    "How do I transfer money to another account",
    "My card is about to expire"  # This was a rare class!
]

# Move model to GPU if available
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
model.to(device)

for i, sample in enumerate(test_samples):
    inputs = tokenizer(sample, return_tensors="pt", padding=True, truncation=True)
    # Move input tensors to the same device as the model
    inputs = {k: v.to(device) for k, v in inputs.items()}

    with torch.no_grad():
        outputs = model(**inputs)
        probabilities = torch.nn.functional.softmax(outputs.logits, dim=-1)

    predicted_class = outputs.logits.argmax(-1).item()
    confidence = probabilities.max().item()
    class_name = id2label[predicted_class]

    print(f"\n{i+1}. '{sample}'")
    print(f"   → {class_name} (confidence: {confidence:.3f})")

    # Show top 3 predictions for rare classes
    if confidence < 0.7:  # Low confidence might indicate rare class
        top3_probs, top3_indices = torch.topk(probabilities, 3)
        print(f"   Top 3 alternatives:")
        for j, (prob, idx) in enumerate(zip(top3_probs[0], top3_indices[0])):
            print(f"     {j+1}. {id2label[idx.item()]} ({prob:.3f})")

=== Comprehensive Evaluation ===



📊 Evaluation Results:
  eval_loss: 0.8165
  eval_accuracy: 0.8672
  eval_f1: 0.8664
  eval_runtime: 6.5515
  eval_samples_per_second: 470.1210
  eval_steps_per_second: 29.4590
  epoch: 3.0000

=== Per-Class Performance ===
                                                  precision    recall  f1-score   support

                                activate_my_card       1.00      0.90      0.95        40
                                       age_limit       0.93      0.97      0.95        40
                         apple_pay_or_google_pay       1.00      1.00      1.00        40
                                     atm_support       0.97      0.95      0.96        40
                                automatic_top_up       1.00      0.88      0.93        40
         balance_not_updated_after_bank_transfer       0.66      0.72      0.69        40
balance_not_updated_after_cheque_or_cash_deposit       0.89      0.97      0.93        40
                         beneficiary_not_allowed       

In [10]:
import pickle

# Save model
model_save_path = "/content/drive/My Drive/Colab Notebooks/Week 8 banking77/models/banking77-classifier"
trainer.save_model(model_save_path)

# Save class weights separately (for analysis)
class_weights_path = "/content/drive/My Drive/Colab Notebooks/Week 8 banking77/class_weights.pkl"
with open(class_weights_path, 'wb') as f:
    pickle.dump({
        'class_weights': class_weights,
        'id2label': id2label,
        'label2id': label2id
    }, f)